In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import re 
from geopy.distance import geodesic
from datetime import datetime, timedelta, time, date
import matplotlib.pyplot as plt
# Part 2

In [2]:
# Define the inputs

# L: set of flights
flights_df = pd.read_excel('Assignment_Data/Group_4_P2.xlsx', sheet_name='Flight')#.set_index('Flight no.')
flights_list = flights_df['Flight Number'].to_list() 

# P: set of passenger itineraries
paths = pd.read_excel('Assignment_Data/Group_4_P2.xlsx', sheet_name='Itinerary').set_index('Itin No.')

# P_p: set of passenger itineraries with recapture rate from itinerary p
recapture_p = pd.read_excel('Assignment_Data/Group_4_P2.xlsx', sheet_name='Recapture Rate').set_index(['From Itinerary','To Itinerary'])
recapture_p.rename(columns={'From Itinerary': 'p', 'To Itinerary': 'r'}, inplace=True)

# K: set of aircraft types
aircraft_df = pd.read_excel('Assignment_Data/Group_4_P2.xlsx', sheet_name='Aircraft')
aircraft_df.rename(columns={'Type': 'AC Type'}, inplace=True)
aircraft_df.set_index('AC Type', inplace=True)
aircraft = aircraft_df.to_dict(orient='index')
ac_list = list(aircraft.keys())

# make a dictionary with the itinerary as the key and the rest as a sub-dictionary
paths = paths.to_dict(orient='index')
path_list = list(paths.keys())

# flights = flights_df.to_dict(orient='index')
recapture_p = recapture_p.to_dict(orient='index')

# For all paths if 'Leg 1' and 'Leg 2' are numbers then create a list with both legs else, drop the keys from the list, and create a new key called 'Legs'
# else just change the name of the key 'Leg 1' to 'Legs'
for key in path_list:
    legs = []
    if paths[key]['Leg 1'] != 0 and paths[key]['Leg 2'] != 0:
        legs.append(paths[key]['Leg 1'])
        legs.append(paths[key]['Leg 2'])
        paths[key]['Legs'] = legs
    elif paths[key]['Leg 1'] != 0:
        legs.append(paths[key]['Leg 1'])
        paths[key]['Legs'] = legs
    del paths[key]['Leg 1']
    del paths[key]['Leg 2']

# Define path 999 with a fare of 0 and a demand of 0 
paths[999] = {'Legs': [], 'Demand': 0, 'Fare': 0}

for k in aircraft:
    aircraft[k]['TAT'] = timedelta(minutes=aircraft[k]['TAT'])

flights = flights_df.merge(aircraft_df.reset_index()[['AC Type']], how='cross')

In [3]:
aircraft

{'A330': {'Units': 4, 'Seats': 210, 'TAT': datetime.timedelta(seconds=3000)},
 'A340': {'Units': 2, 'Seats': 220, 'TAT': datetime.timedelta(seconds=3600)},
 'B737': {'Units': 8, 'Seats': 120, 'TAT': datetime.timedelta(seconds=2100)},
 'B738': {'Units': 29, 'Seats': 140, 'TAT': datetime.timedelta(seconds=2100)}}

In [4]:
# Get the cost from the column named after the AC Type
flights['Cost'] = flights.apply(lambda row: row[row['AC Type']], axis=1)
flights.drop(columns=ac_list, inplace=True)
flights['Capacity'] = flights.apply(lambda row: aircraft[row['AC Type']]['Seats'], axis=1)

In [5]:
# Read the data distance file
distance_info = pd.read_csv('Assignment_Data/Group_4_Distances.csv')
distance_info.rename(columns={'Unnamed: 0': 'Origin'}, inplace=True)

# Create a dictionary with the distance info
distance = {}
for index, row in distance_info.iterrows():
    distance[row['Origin']] = row.to_dict()

# Remove the origin column from the distance
for i in distance:
    distance[i].pop('Origin', None)

In [6]:
# List of unique airports from Origin and Destination columns
airports = list(set(flights['ORG'].unique()).union(set(flights['DEST'].unique())))

In [7]:
misc_date = date(1,1,1)

# Drop rows with distance > range
flights['Arrival'] = flights.apply(lambda row: (datetime.combine(misc_date,row['Arrival']) + aircraft[row['AC Type']]['TAT']).time(), axis=1)
flights['Overnight'] = flights.apply(lambda row: row['Arrival'] < row['Departure'], axis=1)

In [8]:
# Make flights dictionary with main keys: AC Type, with a sub dictionary of flight numbers and each with flight details
flights_dict = {}
for i in flights_list:
    flights_dict[i] = flights[flights['Flight Number'] == i].set_index('AC Type').to_dict(orient='index')

flights = flights_dict

In [9]:
import pandas as pd

# Create an empty list to store the data
data = []
# Iterate over airports, aircraft types, and flights
for l in flights:
    for k in flights[l]:
            for n in airports:
                if flights[l][k]['ORG'] == n:
                    data.append([k, n, l, flights[l][k]['Departure'], 'Departure'])
                if flights[l][k]['DEST'] == n:
                    data.append([k, n, l, flights[l][k]['Arrival'], 'Arrival'])

# Create a dataframe with the data
events = pd.DataFrame(data, columns=['AC Type','Airport', 'Flight N', 'Time', 'D_A'])

# Add the TAT to the arrival times

events.sort_values(by=['AC Type', 'Airport', 'Time'], inplace=True)

# Reset the numbering of the events
events.reset_index(drop=True, inplace=True)

In [10]:
# For each airport and aircraft type i need to create a loop of ground arcs, each starting from the last event and ending at the next event, if there is no more events then the last ground arc is the overnight arc and it ends at the first event of the next day (first event of the next day is the first event of the same airport and aircraft type)

ground_arcs = pd.DataFrame(columns=['AC Type', 'Airport', 'Start Time', 'End Time'])

for k in ac_list:
    for n in airports:
        df = events[(events['AC Type'] == k) & (events['Airport'] == n)].sort_values(by=['Time'])
        for i in range(len(df)):
            if i == 0:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)
            else:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[i-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)

ground_arcs.sort_values(by=['AC Type', 'Airport'], inplace=True)
# Drop rows if start time and end time are the same
ground_arcs = ground_arcs[~ground_arcs.apply(lambda row: (row['Start Time'] == row['End Time']), axis=1)]
ground_arcs['Arc ID'] = ground_arcs.groupby(['AC Type']).cumcount()
ground_arcs['Overnight'] = ground_arcs.apply(lambda row: (row['End Time'] < row['Start Time']), axis=1)
ground_arcs.sort_values(by='Arc ID', inplace=True)

In [11]:
# Create a nodes_df
nodes_df = ground_arcs[['AC Type', 'Airport', 'Start Time']].rename(columns={'Start Time': 'Time'})

# Add a count number for each row group by AC Type and Airport
nodes_df['Node ID'] = nodes_df.groupby(['AC Type']).cumcount()

# make a dictionary with the ac type as main key and the airport as secondary key with the node as tertiary key and the time as value
nodes = {}
for k in ac_list:
    nodes[k] = {}
    for n in airports:
        nodes[k][n] = {}
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i] = {'Time': nodes_df[(nodes_df['AC Type'] == k) & 
                                               (nodes_df['Airport'] == n) & 
                                               (nodes_df['Node ID'] == i)]['Time'].values[0]}
            

In [12]:
# Add node id to the events
events.merge(nodes_df, how='left', on=['AC Type', 'Airport', 'Time']).sort_values(by=['AC Type', 'Airport'])

# Add a dictionary call departures and another one arrivals to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['Departures'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Departure')]['Flight N'])
            nodes[k][n][i]['Arrivals'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Arrival')]['Flight N'])


In [13]:
# n+: ground arcs originating at any node n (start time)
# n-: ground arcs ending at any node n (end time)
n_plus = ground_arcs[['Airport', 'AC Type', 'Start Time', 'Arc ID']].rename(columns={'Start Time': 'Time'})
n_minus = ground_arcs[['Airport', 'AC Type', 'End Time', 'Arc ID']].rename(columns={'End Time': 'Time'})

# Add a dictionary call n+ and another one n- to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['n+'] = list(n_plus[(n_plus['AC Type'] == k) & (n_plus['Airport'] == n) & (n_plus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])
            nodes[k][n][i]['n-'] = list(n_minus[(n_minus['AC Type'] == k) & (n_minus['Airport'] == n) & (n_minus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])

In [14]:
nodes

{'A330': {'EDDB': {24: {'Time': datetime.time(22, 20),
    'Departures': ['AR1690'],
    'Arrivals': [],
    'n+': [24],
    'n-': [45]},
   25: {'Time': datetime.time(2, 15),
    'Departures': [],
    'Arrivals': ['AR1667'],
    'n+': [25],
    'n-': [24]},
   26: {'Time': datetime.time(2, 45),
    'Departures': ['AR1668'],
    'Arrivals': [],
    'n+': [26],
    'n-': [25]},
   27: {'Time': datetime.time(9, 35),
    'Departures': [],
    'Arrivals': ['AR1673'],
    'n+': [27],
    'n-': [26]},
   28: {'Time': datetime.time(10, 10),
    'Departures': ['AR1676'],
    'Arrivals': [],
    'n+': [28],
    'n-': [27]},
   29: {'Time': datetime.time(10, 35),
    'Departures': [],
    'Arrivals': ['AR1691'],
    'n+': [29],
    'n-': [28]},
   30: {'Time': datetime.time(10, 55),
    'Departures': ['AR1801', 'AR1802'],
    'Arrivals': [],
    'n+': [30],
    'n-': [29]},
   31: {'Time': datetime.time(12, 30),
    'Departures': [],
    'Arrivals': ['AR1671'],
    'n+': [31],
    'n-': [30]},
 

In [15]:
overnight_arcs = ground_arcs[ground_arcs['Overnight'] == True][['AC Type', 'Airport', 'Arc ID']]
overnight_flights = []
for l in flights:
    for k in flights[l]:
        if flights[l][k]['Overnight']:
            overnight_flights.append([k, l])

overnight_flights = pd.DataFrame(overnight_flights, columns=['AC Type', 'Flight no.'])

In [16]:
flights

{'AR1132': {'A330': {'Flight Number': 'AR1132',
   'ORG': 'LIRF',
   'DEST': 'KJFK',
   'Departure': datetime.time(23, 55),
   'Arrival': datetime.time(12, 0),
   'Cost': 19797.63398491833,
   'Capacity': 210,
   'Overnight': True},
  'A340': {'Flight Number': 'AR1132',
   'ORG': 'LIRF',
   'DEST': 'KJFK',
   'Departure': datetime.time(23, 55),
   'Arrival': datetime.time(12, 10),
   'Cost': 24304.69708224594,
   'Capacity': 220,
   'Overnight': True},
  'B737': {'Flight Number': 'AR1132',
   'ORG': 'LIRF',
   'DEST': 'KJFK',
   'Departure': datetime.time(23, 55),
   'Arrival': datetime.time(11, 45),
   'Cost': 1000000.0,
   'Capacity': 120,
   'Overnight': True},
  'B738': {'Flight Number': 'AR1132',
   'ORG': 'LIRF',
   'DEST': 'KJFK',
   'Departure': datetime.time(23, 55),
   'Arrival': datetime.time(11, 45),
   'Cost': 1000000.0,
   'Capacity': 140,
   'Overnight': True}},
 'AR1133': {'A330': {'Flight Number': 'AR1133',
   'ORG': 'KJFK',
   'DEST': 'LIRF',
   'Departure': datetime.

In [17]:
# s_ip: binary variable indicating whether flight i is in itinerary p
s_ip = {}
for i in flights_list:
    for p in paths:
        s_ip[i,999] = 0
        if i in paths[p]['Legs']:
            s_ip[i,p] = 1
        else:
            s_ip[i,p] = 0

# Q_i: unconstrained demand for flight i = sum s_ip * demand of itinerary p for p in P
Q_i = {}
for i in flights_list:
    Q_i[i] = 0
    for p in paths:
        Q_i[i] += s_ip[i,p] * paths[p]['Demand']

In [18]:
# Add entries to P_p for path 0 with a recapture rate of 1
for p in paths:
    recapture_p[p,999] = {'Recapture rate': 1}
    recapture_p[999,p] = {'Recapture rate': 0}

path_list = list(paths.keys())
flight_list = list(flights.keys())

In [19]:
print = False
if print:
    import random
    import plotly.graph_objects as go

    for k in ac_list:
        df = events[events['AC Type'] == k].sort_values(by=['Time'])
        fig = go.Figure()
        
        # Set marker color based on 'D_A' column
        marker_color = ['red' if d_a == 'Departure' else 'blue' for d_a in df['D_A']]
        
        # Add ground arcs
        ground_arcs_k = ground_arcs[ground_arcs['AC Type'] == k]
        for i, row in ground_arcs_k.iterrows():
            # Generate a random color
            random_color = '#' + ''.join(random.choices('0123456789ABCDEF', k=6))
            fig.add_shape(
                type="line",
                x0=row['Start Time'],
                y0=row['Airport'],
                x1=row['End Time'],
                y1=row['Airport'],
                line=dict(color=random_color, width=2)
            )
        
        fig.add_trace(go.Scatter(
            x=df['Time'], 
            y=df['Airport'], 
            mode='markers+text',
            marker=dict(color=marker_color),
            hovertemplate= '<b>Flight no.</b>: ' + df['Flight N'] + '<br>' ))
        
        
        fig.update_layout(
            title="Ground Arcs for AC Type " + k,
            xaxis_title="Time",
            yaxis_title="Airport",
        )
        fig.show()


In [20]:
initial = [999]

In [21]:
# Define the model
m = gp.Model('IFAM')

# Define the decision variables
# f[i,k] 1 if flight arc i is assigned to aircraft type k, 0 otherwise
f = {}

# Define the decision variables
# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t = {}

# y_ak = number of aircraft of type k on the ground arc a (integer with lowerbound 0)
y = {}

for p in path_list:
    for r in initial:
        t[p,r] = m.addVar(vtype=GRB.CONTINUOUS, name='t_'+str(p)+'_'+str(r))

for i in flights:
    for k in flights[i]:
        f[i, k] = m.addVar(vtype=GRB.CONTINUOUS, name='f_' + str(i) + '_' + str(k))

# y_ak = number of aircraft of type k on the ground arc a (integer with lowerbound 0)
y = {}
for k in ac_list:
    for a in list(ground_arcs[(ground_arcs['AC Type'] == k)]['Arc ID']):
        y[a, k] = m.addVar(vtype=GRB.CONTINUOUS, name='y_' + str(a) + '_' + str(k))

m.update()
of = gp.quicksum((paths[p]['Fare'] - recapture_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * 
                  t[p,r] 
                  for r in initial for p in path_list)

of -= gp.quicksum(flights[i][k]['Cost'] *     # c_ik 
                    f[i,k]                       # Binary variable
                    for i in flights for k in flights[i])

# Define the objective function
m.setObjective(of, GRB.MINIMIZE)


# Define the constraints
# Constraint 1: Each flight is assigned to exactly one aircraft type
for i in flights:
    m.addConstr((gp.quicksum(f[i,k] for k in flights[i]) == 1), name='one_ac')

# Constraint 2: The number of AC arriving = AC departing, for each type at each node
# y_n+_k + sum(f_i,k) = y_n-_k + sum(f_i,k)
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            m.addConstr((y[nodes[k][n][i]['n+'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Departures']) == 
                         y[nodes[k][n][i]['n-'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Arrivals']) ),
                         name='balance_' + str(i) + '_' + str(k) + '_' + str(n))

# Constraint 3: The number of overnight arcs + the number of overnight flights = the number of aircraft of each type 
# using overnight_arcs and overnight_flights
# sum(y_a,k) + sum(f_i,k) = number of aircraft of type k
for k in ac_list:
    m.addConstr((gp.quicksum(y[a, k] for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID'])) + 
                 gp.quicksum(f[i,k] for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.'])) == 
                 aircraft[k]['Units']), name='overnight_' + str(k))

# Constraint 4: # sum seats_k * f_ik -sum s_ip * t_pr - sum sum s_ip * brp * t_rp >= ds_i for all i but for r = 0 
m.addConstrs((gp.quicksum(aircraft[k]['Seats'] * f[i,k] for k in flights[i]) +
              gp.quicksum(s_ip[i,p] * t[p,r] for p in path_list for r in initial) - 
              gp.quicksum(s_ip[i,p] * recapture_p[(r,p)]['Recapture rate'] * t[p,r] for p in path_list for r in initial) >= 
              Q_i[i] for i in flight_list), name='π')

# Constraint 5: sum t_pr <= Dp for all p
m.addConstrs((t[p,r] <= paths[p]['Demand'] for p in path_list for r in initial), name='σ')


m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-19


In [22]:
# Optimize the model but dont print the output
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2467 rows, 2879 columns and 7996 nonzeros
Model fingerprint: 0x217826a7
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e+01, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+02]
Presolve removed 1538 rows and 1028 columns
Presolve time: 0.01s
Presolved: 929 rows, 1851 columns, 5215 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.1744507e+34   1.093750e+30   1.174451e+04      0s
    1151   -1.7007221e+07   0.000000e+00   0.000000e+00      0s

Solved in 1151 iterations and 0.05 seconds (0.03 work units)
Optimal objective -1.700722093e+07


In [23]:
m.objVal

-17007220.93350528

In [24]:
# Print the optimal objective function value
print('Optimal Objective Function Value: ' + str(m.objVal))

TypeError: 'bool' object is not callable

In [ ]:
for k in ac_list:
    for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID']):
        print(a, k, y[a, k].varName, y[a, k].x)
    for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.']):
        print(i, k, f[i,k].varName, f[i,k].x) 
    print(aircraft[k]['Units'])
    print('\n')
    

TypeError: 'bool' object is not callable

In [ ]:
print('Flights assigned to aircraft type: A320')
for i in flights:
    for k in flights[i]:
        if f[i,k].x > 0 and k == 'A320':
            print(i," | ", ('Cost = %0.2f' % (aircraft[k]['CASK'] *  aircraft[k]['Seats'] *  flights[i][k]['Distance'] * f[i,k].x / 1000)))
print('\n')

print('Flights assigned to aircraft type: A310')
for i in flights:
    for k in flights[i]:
        if f[i,k].x > 0 and k == 'A310':
            print(i," | ", ('Cost = %0.2f ' % (aircraft[k]['CASK'] *  aircraft[k]['Seats'] *  flights[i][k]['Distance'] * f[i,k].x / 1000)))       

print('\n')
# Count the non null values of f
total_flights = 0
for v in m.getVars():
    if v.x != 0 and v.varName[0] == 'f':
        total_flights += 1
print('Total flights: ' + str(total_flights))


Flights assigned to aircraft type: A320
S4220  |  Cost = 1102.18
S4124  |  Cost = 1102.18
S4128  |  Cost = 1102.18
S4320  |  Cost = 1158.76
S4212  |  Cost = 1158.76
S4121  |  Cost = 1102.18
S4125  |  Cost = 1102.18
S4129  |  Cost = 1102.18
S4321  |  Cost = 1158.76
S4213  |  Cost = 1158.76


Flights assigned to aircraft type: A310
S4120  |  Cost = 2353.96 
S4160  |  Cost = 1570.92 
S4222  |  Cost = 7322.90 
S4221  |  Cost = 6264.35 
S422Y  |  Cost = 2353.96 
S4161  |  Cost = 1570.92 
S4223  |  Cost = 7322.90 
S422X  |  Cost = 6264.35 


Total flights: 18


In [ ]:
# Make a df with AC grounded at overnight on each airport
grounded_overnight = pd.DataFrame(columns=ac_list, index=airports)
overnight_arcs['count'] = overnight_arcs.apply(lambda row: y[row['Arc ID'], row['AC Type']].x, axis=1)

# Reorganize the df, make the columns the unique AC types and the rows the unique airports and the values the count
for k in ac_list:
    grounded_overnight[k] = overnight_arcs[overnight_arcs['AC Type'] == k].groupby(['Airport']).sum()['count']

# Fill the NaN values with 0
grounded_overnight.fillna(0, inplace=True)
grounded_overnight

,A310,A320
BOS,1.0,0.0
OPO,0.0,1.0
YTO,0.0,0.0
PDL,1.0,1.0
FNC,0.0,0.0
LIS,1.0,1.0


In [ ]:
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            # val = 0
            s = str(i) + " | " + str(k) + " | "  + str(n)+ " | " +str(nodes[k][n][i]['Time']) + " | " 
            s += str(y[nodes[k][n][i]['n+'][0], k].varName) + ' (n+)'
            # val += y[nodes[k][n][i]['n+'][0], k].varName
            for w in nodes[k][n][i]['Departures']:
                s += ' + ' + str(f[w, k].varName) + ' (f+)'
                # val += f[w, k].varName
            s += ' = ' + str(y[nodes[k][n][i]['n-'][0], k].varName)  + ' (n-)'
            # val -= y[nodes[k][n][i]['n-'][0], k].varName
            for w in nodes[k][n][i]['Arrivals']:
                s += ' + ' + str(f[w, k].varName) + ' (f-)'
                # val -= f[w, k].varName
            # s += ' = ' + str(val)
            print(s)

0 | A310 | BOS | 22:25:00 | y_0_A310 (n+) = y_1_A310 (n-) + f_S4221_A310 (f-)
1 | A310 | BOS | 03:15:00 | y_1_A310 (n+) + f_S422X_A310 (f+) = y_0_A310 (n-)
11 | A310 | OPO | 21:50:00 | y_11_A310 (n+) = y_14_A310 (n-) + f_S4212_A310 (f-)
12 | A310 | OPO | 12:25:00 | y_12_A310 (n+) = y_11_A310 (n-) + f_S4320_A310 (f-)
13 | A310 | OPO | 13:30:00 | y_13_A310 (n+) + f_S4321_A310 (f+) = y_12_A310 (n-)
14 | A310 | OPO | 19:05:00 | y_14_A310 (n+) + f_S4213_A310 (f+) = y_13_A310 (n-)
31 | A310 | YTO | 02:45:00 | y_31_A310 (n+) + f_S4223_A310 (f+) = y_32_A310 (n-)
32 | A310 | YTO | 00:25:00 | y_32_A310 (n+) = y_31_A310 (n-) + f_S4222_A310 (f-)
15 | A310 | PDL | 22:05:00 | y_15_A310 (n+) + f_S4128_A310 (f+) = y_30_A310 (n-) + f_S4213_A310 (f-)
16 | A310 | PDL | 08:40:00 | y_16_A310 (n+) = y_15_A310 (n-) + f_S422X_A310 (f-)
17 | A310 | PDL | 09:00:00 | y_17_A310 (n+) = y_16_A310 (n-) + f_S4223_A310 (f-)
18 | A310 | PDL | 09:25:00 | y_18_A310 (n+) + f_S4120_A310 (f+) = y_17_A310 (n-) + f_S4121_A310

In [ ]:
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            val = 0
            s = str(i) + " | " + str(k) + " | "  + str(n)+ " | " +str(nodes[k][n][i]['Time']) + " | " 
            s += str(y[nodes[k][n][i]['n+'][0], k].x) + ' (n+)'
            val += y[nodes[k][n][i]['n+'][0], k].x
            for w in nodes[k][n][i]['Departures']:
                s += ' + ' + str(f[w, k].x) + ' (f+)'
                val += f[w, k].x
            s += ' = ' + str(y[nodes[k][n][i]['n-'][0], k].x)  + ' (n-)'
            val -= y[nodes[k][n][i]['n-'][0], k].x
            for w in nodes[k][n][i]['Arrivals']:
                s += ' + ' + str(f[w, k].x) + ' (f-)'
                val -= f[w, k].x
            s += ' | Total ' + str(val)
            print(s)

0 | A310 | BOS | 22:25:00 | 1.0 (n+) = 0.0 (n-) + 1.0 (f-) | Total 0.0
1 | A310 | BOS | 03:15:00 | 0.0 (n+) + 1.0 (f+) = 1.0 (n-) | Total 0.0
11 | A310 | OPO | 21:50:00 | 0.0 (n+) = 0.0 (n-) + 0.0 (f-) | Total 0.0
12 | A310 | OPO | 12:25:00 | 0.0 (n+) = 0.0 (n-) + -0.0 (f-) | Total 0.0
13 | A310 | OPO | 13:30:00 | 0.0 (n+) + -0.0 (f+) = 0.0 (n-) | Total 0.0
14 | A310 | OPO | 19:05:00 | 0.0 (n+) + 0.0 (f+) = 0.0 (n-) | Total 0.0
31 | A310 | YTO | 02:45:00 | -0.0 (n+) + 1.0 (f+) = 1.0 (n-) | Total 0.0
32 | A310 | YTO | 00:25:00 | 1.0 (n+) = -0.0 (n-) + 1.0 (f-) | Total 0.0
15 | A310 | PDL | 22:05:00 | 1.0 (n+) + -0.0 (f+) = 1.0 (n-) + 0.0 (f-) | Total 0.0
16 | A310 | PDL | 08:40:00 | 2.0 (n+) = 1.0 (n-) + 1.0 (f-) | Total 0.0
17 | A310 | PDL | 09:00:00 | 3.0 (n+) = 2.0 (n-) + 1.0 (f-) | Total 0.0
18 | A310 | PDL | 09:25:00 | 2.0 (n+) + 1.0 (f+) = 3.0 (n-) + -0.0 (f-) | Total 0.0
19 | A310 | PDL | 09:35:00 | 2.0 (n+) + -0.0 (f+) = 2.0 (n-) | Total 0.0
20 | A310 | PDL | 10:35:00 | 1.0 (n+)